# setup

In [11]:
import warnings
warnings.filterwarnings('ignore')

import os
import scipy
import numpy as np
import nibabel as nib
from nilearn import image
from brainiak.searchlight.searchlight import Searchlight
from brainiak.searchlight.searchlight import Cube

import analysis_helpers as helpers

In [ ]:
# root paths
data_dir = '/jukebox/ntb/projects/sketchloop02/data'
SAVE_PATH = '/jukebox/ntb/projects/sketchloop02/data/searchlight_output'

# module definitions
roi_dir = os.path.join(data_dir, 'copes/roi')
cope_dir = os.path.join(data_dir, 'copes/recog/objectGLM')
sub_dirs = sorted(os.listdir(roi_dir))
helpers.roi_dir, helpers.cope_dir = roi_dir, cope_dir

# null mask
null_mask = np.ones((88, 128, 128))

In [13]:
# need version of this function that gives access to affine
def load_single_run_weights(subj,run_num,cope_num):
    nifti_path = os.path.join(cope_dir, subj[:7] + '_run' + str(run_num) + '_cope' + str(cope_num) + '_hires.nii.gz')
    fmri_img = image.load_img(nifti_path)
    fmri_data = fmri_img.get_data()
    return fmri_data, fmri_img.affine

# need version of this function that resolves presence of nans in matrix
def pairwise_distances(matrix):
    output = np.zeros((8, 8))
    for i in range(8):
        for j in range(8):
            a = matrix[i, :]
            b = matrix[j, :]
            notnan = np.logical_and((b != np.nan),(a != np.nan))
            output[i,j] = scipy.spatial.distance.correlation(a[notnan], b[notnan])
    return output

# define and run searchlight
How searchlight works with brainiak:

1. Initiate a searchlight object, articulating certain parameters (e.g., searchlight shape, radius, max edge length, in voxels, of the 3D block(?)).
2. Distribute data to be searched to the searchlight object, sorting between MPI ranks (idk what MPI ranks are tbh).
3. Broadcast data, i.e., define other variables to be available for each execution of the searchlight function.
4. Run the searchlight, this time articulating as a parameter the function to be applied at each searchlight location.

We want to perform searchlight again, but this time the objective is to perform a version of helpers.make_drawing_predictions on each searchlight mask.

In [14]:
def prepostRSA(subject_data, mask, sl_rad, bcast_var):
    # first PRE
    mat1 = np.array([subject_data[0][:, :, :, i].ravel() for i in range(4)])
    mat2 = np.array([subject_data[0][:, :, :, i].ravel() for i in range(4,8)])
    fAB = np.vstack((mat1,mat2)) # stack feature matrices
    DAB = pairwise_distances(fAB, metric='correlation') # pre
    offblock = DAB[:len(mat1),range(len(mat1),np.shape(DAB)[1])]

    trained_witobj = offblock.diagonal()[:2]
    control_witobj = offblock.diagonal()[2:]
    trained_btwobj = np.array([offblock[:2,:2][0,1], offblock[:2,:2][1,0]])
    control_btwobj = np.array([offblock[2:,2:][0,1],offblock[2:,2:][1,0]])

    trawit_mean = np.nanmean(trained_witobj)
    conwit_mean = np.nanmean(control_witobj)
    trabtw_mean = np.nanmean(trained_btwobj)
    conbtw_mean = np.nanmean(control_btwobj)
    tradiffpre = trabtw_mean - trawit_mean
    condiffpre = conbtw_mean - conwit_mean
    
    # now POST
    mat1 = np.array([subject_data[0][:, :, :, i].ravel() for i in range(8,12)])
    mat2 = np.array([subject_data[0][:, :, :, i].ravel() for i in range(12,16)])
    fAB = np.vstack((mat1,mat2)) # stack feature matrices|
    DAB = pairwise_distances(fAB, metric='correlation') # post
    offblock = DAB[:len(mat1),range(len(mat1),np.shape(DAB)[1])]

    trained_witobj = offblock.diagonal()[:2]
    control_witobj = offblock.diagonal()[2:]
    trained_btwobj = np.array([offblock[:2,:2][0,1], offblock[:2,:2][1,0]])
    control_btwobj = np.array([offblock[2:,2:][0,1],offblock[2:,2:][1,0]])

    trawit_mean = np.nanmean(trained_witobj)# .mean()
    conwit_mean = np.nanmean(control_witobj)#.mean()
    trabtw_mean = np.nanmean(trained_btwobj)#.mean()
    conbtw_mean = np.nanmean(control_btwobj)#.mean()
    tradiffpost = trabtw_mean - trawit_mean
    condiffpost = conbtw_mean - conwit_mean
    
    return (tradiffpost-tradiffpre)-(condiffpost-condiffpre)

In [15]:
# function that sets up and organizes searchlight over a set of subjects
def searchlight_over_each(subjects, sl_rad):
    
    for s in subjects:
        print(s)
        
        # set up searchlight object
        sl = Searchlight(sl_rad=sl_rad, shape=Cube)
        
        # arrange data to be distributed to searchlight
        # list of 4D not 3D arrays
        subject_data = []
        
        for run in [3, 4, 5, 6]:
            condorder = helpers.get_condorder(s)
            for c in condorder:
                weights, affine = load_single_run_weights(s, run, c)
                subject_data.append(weights)

        #subject_data = [s.reshape((88, 128, 128, 1)) for s in subject_data]
        subject_data = [np.stack(subject_data, axis=3)]
        
        # distribute and broadcast needed data to searchlight
        sl.distribute(subject_data, null_mask)
        sl.broadcast(None)
        
        # run searchlight
        subject_outputs = np.array(sl.run_searchlight(prepostRSA))
        print(np.shape(subject_outputs.astype(np.float32)))
        img = nib.Nifti1Image(subject_outputs.astype(np.float32), affine)
        # store output for this subject
        if not os.path.exists(SAVE_PATH):
            os.makedirs(SAVE_PATH)
        
        localizer_tmap_filename = os.path.join(SAVE_PATH, s + 'prepost_searchlight.nii.gz')
        nib.save(img, localizer_tmap_filename)

In [ ]:
# run searchlight over all subjects and print time to compute results
searchlight_over_each(sub_dirs, 5)